In [1]:
# Import and initialize Google Earth Engine.
import ee
ee.Initialize()

In [2]:
# Import relevant packages and functions.

%matplotlib notebook

from __future__ import print_function # For py 2.7 compat


import datetime
import ipywidgets
from IPython.display import display
from IPython.display import Image
import traitlets
from IPython.core.display import Javascript

In [3]:
import IPython
IPython.__version__

'5.1.0'

In [55]:
# Enter user inputs area of interest, associated water lines, 
# storm duration, and storm dates.
studyArea = ee.FeatureCollection( 
    'ft:1LDn3ycFakx0QZltmUz3ZurEhmbC9snAnXc6b-5zR').filter(ee.Filter.eq(
        'reg', 'DAKAR'))
waterLines = ee.FeatureCollection( 
    'ft:1earShoNRTjW-FiXA5ESgk9N3kcqsh_XCAN1eq7lN')
stormDuration = 8
stormStartDate = ee.Date('2012-08-28')
stormEndDate = ee.Date('2012-09-04')
yearStartDate = ee.Date('2012-01-01')
yearEndDate = ee.Date('2012-12-31')
dfo = ee.FeatureCollection( 
    'ft:1z-JcOwKYuxL1YoWQH4k7ByI75-2kzT91ctz286BI')
dfo3180 = ee.Image(
    'users/jonathanasullivan/geeOtsuDFOAssets_Senegal/Senegal_Otsu_DFO3180_3Day')
mod44 = ee.Image('MODIS/MOD44W/MOD44W_005_2000_02_24')
hansen = ee.Image('UMD/hansen/global_forest_change_2015')



In [11]:
# Import the Digital Elevation Model (DEM).
elevation = ee.Image("USGS/SRTMGL1_003")

In [12]:
# Local slope - This calculates slope in degrees from the DEM.
slope = ee.Terrain.slope(elevation)

In [13]:
# Curvature - This calculates the curvature, which is defined as the 
# slope of the slope. You can think of it as the shape (
# convex, concave, flat) of the area. While slope helps define the 
# rate of of run-off, curvature helps determine the direction of flow. 
curv = (elevation.convolve(ee.Kernel.laplacian8())).resample()


In [14]:
# Section 2.2 Climate & Precipitation Indicators

# Total precipitation during the flood event:
precip = ee.Image(ee.ImageCollection('NOAA/PERSIANN-CDR')
                  .filterDate(stormStartDate,stormEndDate)
                  .select("precipitation").sum())


In [15]:
# Define event precipitation intensity as Pi = (Sm + Am) / (Am – Sm).

# Maximum 24-hr precipitation during the flood event:
stormMax = ee.Image(ee.ImageCollection('NOAA/PERSIANN-CDR')
                    .filterDate(stormStartDate,stormEndDate)
                    .select("precipitation").max())

# Maximum 24-hr precipitation for the year of the flood event:
annualMax = ee.Image(ee.ImageCollection('NOAA/PERSIANN-CDR')
                     .filterDate(yearStartDate, yearEndDate)
                     .select("precipitation").max())

pi = (stormMax.add(annualMax)).divide(annualMax.subtract(stormMax))

In [56]:
# Import land surface temperature 
temp = ee.ImageCollection('MODIS/MYD11A1').filterDate(
    stormStartDate,stormEndDate).select('LST_Day_1km')

In [17]:
# Section 2.3 Development Indicators
 
# Impervious surface  -bring in land cover/land use data
cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0)
imp = cover.eq(190)

In [23]:
# Section 2.4 Hydrologic Indicators 

# Euclidean distance from water features 
# Select waterfeatures by filtering for LC labeled 
# 180 (GlobCover category for H20).

waterf= ee.Image("MODIS/MOD44W/MOD44W_005_2000_02_24").select(
    "water_mask");
cover = ee.Image('ESA/GLOBCOVER_L4_200901_200912_V2_3').select(0);
waterf = cover.eq(210)
imp = cover.eq(190)

In [24]:
# Create a distance kernel - defining area of analysis.
kern = ee.Kernel.euclidean(2000, 'meters')

In [25]:
# Associate the viewing distance (kern) with the water features
distance = waterf.distance(kern)

In [26]:
# Calculate distance from rivers using the NHD data 
# and the distance variable
riverDist = waterLines.distance().clip(studyArea)

In [27]:
# Topographic Wetness Index (TWI):
# Preprocessing includes generating flow accumulation, 
# catchment area & slope --15 sec flow accumulation.
flowAcc = ee.Image('WWF/HydroSHEDS/15ACC')

In [28]:
# Calculate flow accumulation in the flood zone.
floodFlow = flowAcc.clip(studyArea)

In [29]:
# Create layer of just 1's. 
imageof1 = ee.Image.constant(1)

In [30]:
# Add +1 to flow accumulation.
# This is so you don't get zeros when you take the log.
floodFlowPlus1 = floodFlow.add(imageof1)

In [32]:
# Pixel dimensions of flow accumulation:
pixelArea = ee.Image.pixelArea().mask(floodFlow.mask())

In [33]:
# Now calculate the upstream area for each pixel using 
# contributing catchment area according to topmodel 
# topographic index equation.
# CA= (flow accum + 1 * cell^2)
# This is calculated by multiplying the number of pixels 
# flowing into a given pixel * the area. 
ca = floodFlowPlus1.multiply(pixelArea)

In [34]:
# Add a very low number to the slope so that zero.
# Slopes don't get counted out of the index.
imageofalmost0 = ee.Image.constant(0.0000001)

In [35]:
# Get new slope with no zero.
nonzeroSlopes = slope.add(imageofalmost0)

In [37]:
# Convert to radians.
slopeRadians = nonzeroSlopes.multiply(3.141593/180)

In [38]:
# Calcualte the tan of the slope for TI equation.
tanSlope = slopeRadians.tan()
clipped2 = tanSlope

In [39]:
# The topographic index is LN([flowaccum+1*cellarea]/tanslope)
# It is defined as ln(a/tanβ) where a is the local
# upslope area draining through a certain point per unit contour
# length and tanβ is the local slope.
# Note that both TWI and SPI assume steady state.
topoIndex = ca.divide(clipped2)
twi = topoIndex.log()

In [40]:
# Stream Power Index (SPI)
# SPI is CA/slope (Gallant 2000 Terrain Analysis).
streamPower = ca.multiply(clipped2) #final SPI
spi = streamPower.clip(studyArea) #clip to watershed

In [41]:
# Aspect (snowmelt): 
# This tells you the direction of the slope (0-360 degrees)
# which is important for snowmelt.
aspect = ee.Terrain.aspect(elevation)

In [42]:
# Height Above Nearest Drainage:
hand = ee.ImageCollection('users/gena/global-hand/hand-100').mosaic()

In [57]:
def radians(img):
    return img.toFloat().multiply(3.1415927).divide(180)

def hillshade(az, ze, slope, aspect):
    azimuth = radians(ee.Image(az))
    zenith = radians(ee.Image(ze))
    return azimuth.subtract(aspect).cos().multiply(
        slope.sin()).multiply(zenith.sin()).add(zenith.cos().multiply(
            slope.cos()))

def hillshadeit(image, elevation, weight, height_multiplier, 
                azimuth, zenith):
    hsv  = image.unitScale(0, 255).rgbtohsv()
    
    terrain = ee.call('Terrain', elevation.multiply(height_multiplier))
    slope = radians(terrain.select(['slope']))

    aspect = radians(terrain.select(['aspect'])).resample('bicubic')
    hs = hillshade(azimuth, zenith, slope, aspect).resample('bicubic')

    intensity = hs.multiply(weight).multiply(hsv.select('value'));
    huesat = hsv.select('hue', 'saturation');

    return ee.Image.cat(huesat, intensity).hsvtorgb()

In [47]:
# Global DEM mosaic: 
DEM1 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM1')
DEM2 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM2')
DEM3 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM3')
DEM4 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM4')
DEM5 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM5')
DEM6 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM6')
DEM7 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM7')
DEM8 = ee.Image('users/gena/ViewfinderpanoramaDEM/VFP_DEM8')

dem = ee.Image.cat(DEM1,DEM2,DEM3,DEM4,DEM5,DEM6,DEM7,DEM8,elevation)


In [48]:
azimuth = 90
zenith = 20
hsWeight = 1.1
heightMultiplier = 4

In [50]:
# Building a water mask
# These are waterbodies.
mod44Water = mod44.select([0]).clip(studyArea)
hansenW = hansen.select('datamask').eq(2)
hansenWater = hansenW.clip(studyArea)

In [51]:
# This masks out water pixels.
hansen = hansen.select('datamask')
hansenLand = hansen.select([0]).remap([0,1,2], [1,1,0])

In [52]:
# This generates a multiband image of all the training data.
region =studyArea
variables = (elevation
  .addBands(slope)
  .addBands(curv)
  .addBands(pi)
  .addBands(imp)
  .addBands(precip)
  .addBands(twi)
  .addBands(spi)
  .addBands(riverDist)
  .addBands(aspect)
  .addBands(hand)
  .addBands(temp));

In [53]:
# Mask out pixels that are permanent water bodies
variables = variables.mask(hansen)

In [54]:
# Create variable to store band names
bands = ['elevation','slope', 'elevation_1',
         'precipitation', 'landcover', 'precipitation_1', 
         'b1', 'b1_1', 'distance', 'aspect', 'b1_2', 'temp']